In [10]:
# 接続詞の分類 https://bunsyou.net/kouza1/08setsuzokushi.html
conjunction_category = {
    "順接": ["だから", "それで", "ゆえに", "そこで", "すると", "したがって", "よって"],
    "逆接": ["が", "だが", "しかし", "けれど", "けれども", "だけど", "ところが", "とはいえ", "それでも"],
    "添加": ["そして", "それから", "また", "しかも", "その上", "さらに", "なお", "かつ", "および"],
    "対比": ["または", "あるいは", "それとも", "そのかわり", "むしろ", "ないしは", "いっぽう", "もしくは"],
    "転換": ["さて", "ところで", "では", "それでは", "次に", "ときに"],
    "補足": ["つまり", "すなわち", "なぜなら", "たとえば", "ただし", "ちなみに", "要するに", "いわば"],
}

In [11]:
import re
import glob

all_conjunction_first = 0
all_conjunction_middle = 0

for filename in glob.glob('data/*.txt'):
    with open(filename, 'r', encoding='UTF-8') as f:
        data = f.read()

    # 文ごとに分割するための正規表現パターン
    sentence_pattern = re.compile(r'[^。！？]*[。！？]')

    # 文のリストを作成
    sentences = sentence_pattern.findall(data)


    for i, sentence in enumerate(sentences):
        if "配点" in sentence:
            sentences = sentences[i:]
            break

    conjunction_first = 0
    conjunction_middle = 0
    
    for category, conjunctions in conjunction_category.items():
        for conjunction in conjunctions:
            for i, sentence in enumerate(sentences):
                # 接続詞が文頭にある場合
                if sentence.startswith(conjunction):
                    
                    # 前後の文を取り出す
                    before_conjunction = sentences[max(0, i-2):i]  # 2文前と1文前
                    after_conjunction = [sentences[i][len(conjunction)+1:]] + sentences[i+1:i+2] # 現在の文と次の文
                    conjunction_first += 1
                    all_conjunction_first += 1
                
                # 接続詞が文中にある場合
                elif f"、{conjunction}" in sentence:
                    # conjunctionがsentenceの何文字目かをカウント
                    pos = sentence.find(f"、{conjunction}")
                    if pos != -1:
                        # 前後の文を取り出す
                        before_conjunction = sentences[max(0, i-1):i] + [sentence[:pos]]  # 1文前とconjunction直前までの文
                        after_conjunction = [sentence[pos+1+len(conjunction):]] + sentences[i+1:i+2]  # conjunction直後以降の文と次の文

                        conjunction_middle += 1
                        all_conjunction_middle += 1
                        
    print(f"{filename[5:-4]} の接続詞: 文頭{conjunction_first}回，文中{conjunction_middle}回")
print(f"すべての接続詞: 文頭{all_conjunction_first}回，文中{all_conjunction_middle}回")

2022gendai の接続詞: 文頭15回，文中6回
2023gendai の接続詞: 文頭18回，文中7回
2024gendai の接続詞: 文頭8回，文中2回
「骨太の方針」 持続的な賃上げと財政健全化は の接続詞: 文頭4回，文中0回
アメリカ大統領選挙　トランプ氏“有罪評決”の影響は の接続詞: 文頭1回，文中4回
アメリカ早期の利下げ慎重に その影響と見通しは の接続詞: 文頭0回，文中2回
イスラエル・ハマス戦闘　停戦への合意は可能か？ の接続詞: 文頭6回，文中3回
ロシア戦術核演習と高まる緊張 の接続詞: 文頭8回，文中1回
危険エリアの“逆線引き”は進むか の接続詞: 文頭4回，文中0回
政府の秘密増えるの？増えないの？ 経済安全保障に必要なセキィリティクリアランス制度の課題 の接続詞: 文頭9回，文中3回
欧州議会選挙・右派躍進　欧州の行方は の接続詞: 文頭3回，文中2回
衛星打ち上げ失敗の北朝鮮 キム総書記の視線の先は？ の接続詞: 文頭5回，文中0回
すべての接続詞: 文頭81回，文中30回


In [12]:
# 接続詞の分類 https://bunsyou.net/kouza1/08setsuzokushi.html
conjunction_category = {
    0: ["だから", "それで", "ゆえに", "そこで", "すると", "したがって", "よって"], # 順接
    1: ["が", "だが", "しかし", "けれど", "けれども", "だけど", "ところが", "とはいえ", "それでも"], # 逆接
    2: ["そして", "それから", "また", "しかも", "その上", "さらに", "なお", "かつ", "および"], # 添加
    3: ["または", "あるいは", "それとも", "そのかわり", "むしろ", "ないしは", "いっぽう", "もしくは"], # 対比
    4: ["さて", "ところで", "では", "それでは", "次に", "ときに"], # 転換
    5: ["つまり", "すなわち", "なぜなら", "たとえば", "ただし", "ちなみに", "要するに", "いわば"], # 補足
}

import re
import pandas as pd

# 接続詞の前後4文を取り出す
# 例(C=2)：今日はいい天気だ。散歩に行こう。【または】、遊園地に行こう。さあ着替えて！（文頭のケースでは，前後2文）
# 例(C=2)：今日はいい天気だ。散歩に行くか、【または】、遊園地に行こう。早く支度をしてね。（文中のケースでは，前後1文+接続詞の前後）

def extract_sentence_around_conjunction(filename, conjunction_category=conjunction_category):
    header = ["conjunction", "category", "position", "b4", "b3", "b2", "b1", "a1", "a2", "a3", "a4"]
    df_list = [header]
    
    with open(f'data/{filename}.txt', 'r', encoding='UTF-8') as f:
        data = f.read()

    # 文ごとに分割するための正規表現パターン
    sentence_pattern = re.compile(r'[^。！？]*[。！？]')

    # 文のリストを作成
    sentences = sentence_pattern.findall(data)

    for i, sentence in enumerate(sentences):
        if "配点" in sentence:
            sentences = sentences[i:]
            break
        
        for category, conjunctions in conjunction_category.items():
            for conjunction in conjunctions:
                for i, sentence in enumerate(sentences):
                    each_data_list = []
                    # 接続詞が文頭にある場合
                    if sentence.startswith(conjunction):
                        
                        # 前後の文を取り出す
                        before_conjunction = sentences[max(0, i-4):i]
                        after_conjunction = [sentences[i][len(conjunction)+1:]] + sentences[i+1:i+4]
                        
                        # 欠損している前後の分を空文字列で補完
                        before_conjunction = [""] * (4 - len(before_conjunction)) + before_conjunction
                        after_conjunction = after_conjunction + [""] * (4 - len(after_conjunction))
                    
                        # header = ["conjunction", "category", "position", "b4", "b3", "b2", "b1", "a1", "a2", "a3", "a4"]
                        each_data_list.extend([conjunction, category, "first"] + before_conjunction + after_conjunction)
                        df_list.append(each_data_list)
                        
                    # 接続詞が文中にある場合
                    elif f"、{conjunction}" in sentence:
                        # conjunctionがsentenceの何文字目かをカウント
                        pos = sentence.find(f"、{conjunction}")
                        if pos != -1:
                            # 前後の文を取り出す
                            before_conjunction = sentences[max(0, i-3):i] + [sentence[:pos]]
                            after_conjunction = [sentence[pos+1+len(conjunction):]] + sentences[i+1:i+4]
                            
                        # 欠損している前後の分を空文字列で補完
                        before_conjunction = [""] * (4 - len(before_conjunction)) + before_conjunction
                        after_conjunction = after_conjunction + [""] * (4 - len(after_conjunction))
                    
                        # header = ["conjunction", "category", "position", "b4", "b3", "b2", "b1", "a1", "a2", "a3", "a4"]
                        each_data_list.extend([conjunction, category, "middle"] + before_conjunction + after_conjunction)
                    
                        df_list.append(each_data_list)
                    
    # データフレームに変換して保存する
    df = pd.DataFrame(df_list[1:], columns=df_list[0])  # ヘッダーを除いてデータフレームを作成
    df.to_csv(f'sentence_file/{filename}_info.csv', index=False, encoding='utf-8-sig')  # エンコーディングをUTF-8-SIGに指定して保存

In [13]:
import os
# dataフォルダ内の全ファイルに対して適用
data_folder = 'data'
for filename in os.listdir(data_folder):
    if filename.endswith('.txt'):
        extract_sentence_around_conjunction(filename[:-4])

In [14]:
pd.read_csv('sentence_file/2022gendai_info.csv', encoding='utf-8')

,conjunction,category,position,b4,b3,b2,b1,a1,a2,a3,a4
0,だが,1,first,妖怪は「表象」という人工物へと作り変えられたことによって、人間の手で自由自在にコントロールさ...,こうしたＣ妖怪の「表象」化は、人間の支配力が世界のあらゆる局面、あらゆる「物」に及ぶようにな...,かつて神霊が占めていたその位置を、いまや人―６―間が占めるようになったのである。,１５ここまでが、近世後期より具体的には十八世紀後半以降の都市における妖怪観である。,近代になると、こうした近世の妖怪観はふたたび編成しなおされることになる。,「表象」として、リアリティの領域から切り離されてあった妖怪が、以前とは異なる形でリアリティの...,これは、近世は妖怪をリアルなものとして恐怖していた迷信の時代、近代はそれを合理的思考によって...,けいもう１６「表象」という人工的な記号を成立させていたのは、「万物の霊長」とされた人間の力の...
1,しかし,1,first,NaN,NaN,５０）フィクションとしての妖怪、とりわけ娯楽の対象としての妖怪は、いかなる歴史的背景のもとで...,確かに、鬼や天狗など、古典的な妖怪を題材にした絵画や芸能は古くから存在した。,妖怪が明らかにフィクションてんぐの世界に属する存在としてとらえられ、そのことによってかえって...,つまり、フィクションとしての妖怪という領域自体が歴史性を帯びたものなのである。,妖怪はそもそも、日常的理解を超えた不可思議な現象に意味を与えようとするミンゾク的な心意から生...,人間はつねに、経験に裏打ちされた日常的な原因︱結果の了解に基づいて目の前に生起する現象を認識...
2,しかし,1,middle,つまり、「物」は物そのものと言うよりも「記号」であったのである。,これらの「記号」は所与のものとして存在しており、人間にできるのはその「記号」を「読み取る」こ...,１２「物」が同時に「言葉」を伝える「記号」である世界。,こうした認識は,近世において大きく変容する。,「物」にまとわりついた「言葉」や「記号」としての性質が剝ぎ取られ、はじめて「物」そのものとし...,ここに近世の自然認識や、西洋の博物学に相当する（注）\nほんぞうがく\n本草学という学問が成...,そして妖怪もまた博物学的な思考、あしこうるいは嗜好の対象となっていくのである。
3,しかし,1,first,ここに近世の自然認識や、西洋の博物学に相当する（注）\nほんぞうがく\n本草学という学問が成...,そして妖怪もまた博物学的な思考、あしこうるいは嗜好の対象となっていくのである。,１３この結果、「記号」の位置づけも変わってくる。,かつて「記号」は所与のものとして存在し、人間はそれを「読み取る」ことしかできなかった。,近世においては、「記号」は人間が約束事のなかで作り出すことができるものとなった。,これは、「記号」が神霊の支配を逃れて、人間の完全なコントロール下に入ったことを意味する。,こうした「記号」を、本書では「表象」と呼んでいる。,人工的な記号、人間の支配下にあることがはっきりと刻印された記号、それが「表象」である。
4,ところが,1,first,しかし、妖怪が明らかにフィクションてんぐの世界に属する存在としてとらえられ、そのことによって...,つまり、フィクションとしての妖怪という領域自体が歴史性を帯びたものなのである。,妖怪はそもそも、日常的理解を超えた不可思議な現象に意味を与えようとするミンゾク的な心意から生...,人間はつねに、経験に裏打ちされた日常的な原因︱結果の了解に基づいて目の前に生起する現象を認識...,たま、そうした日常的な因果了解では説明のつかない現象に遭遇する。,それは通常の認識や予見を無効化するため、人間の心に不安と恐怖をカンキする。,このような言わば意味論的な危機に対して、それをなんとか意味の体系のなかに回収するために生み出...,それは人間が秩序ある意味世界のなかで生きていくうえでの必要性から生み出されたものであり、それ...
...,...,...,...,...,...,...,...,...,...,...,...
1381,むしろ,3,middle,これは、「記号」が神霊の支配を逃れて、人間の完全なコントロール下に入ったことを意味する。,こうした「記号」を、本書では「表象」と呼んでいる。,人工的な記号、人間の支配下にあることがはっきりと刻印された記号、それが「表象」である。,１４「表象」は、意味を伝えるものであるよりも,その形象性、視覚的側面が重要な役割を果たす「記号」である。,妖怪は、伝承や説話といった「言葉」の世界、意味の世界から切り離され、名前や視覚的形象によって...,それはまさに、現代で言うところの「キャラクター」であった。,そしてキャラクターとなった妖怪は完全にリアリティを喪失し、フィクショナルな存在として人間の娯...
1382,いっぽう,3,middle,人間は「神経」の作用、「催眠術」の効果、「心霊」の感応によって容易に妖怪を「見てしまう」不安...,かつて「表象」としてフィクショナルな領域に囲い込まれていた妖怪たちは、今度は「人間」そのもの...,そして、こうした認識とともに生み出されたのが、「私」という近代に特有の思想であった。,謎めいた「内面」を抱え込んでしまったことで、「私」は私にとって「不気味なもの」となり,で未知なる可能性を秘めた神秘的な存在となった。,妖怪は、まさにこのような「私」をトウエイした存在としてあらわれるようになるのである。,１８以上がアルケオロジー的方法によって描き出した、妖怪観の変容のストーリーである。,―７―（注）本草学もとは薬用になる動植物などを研究する中国由来の学問で、江戸時代に盛んとなり...
1383,つまり,5,first,NaN,５０）フィクションとしての妖怪、とりわけ娯楽の対象としての妖怪は、いかなる歴史的背景のもとで...,確かに、鬼や天狗など、古典的な妖怪を題材にした絵画や芸能は古くから存在した。,しかし、妖怪が明らかにフィクションてんぐの世界に属する存在としてとらえられ、そのことによって...,フィクションとしての妖怪という領域自体が歴史性を帯びたものなのである。,妖怪はそもそも、日常的理解を超えた不可思議な現象に意味を与えようとするミンゾク的な心意から生...,人間はつねに、経験に裏打ちされた日常的な原因︱結果の了解に基づいて目の前に生起する現象を認識...,ところが時たま、そうした日常的な因果了解では説明のつかない現象に遭遇する。
1384,つまり,5,first,１１中世において、妖怪の出現は多くの場合「凶兆」として解釈された。,それらは神仏をはじめとする神秘的存在からの「警告」であった。,すなわち、妖怪は神霊からの「言葉」を伝えるものという意味で、一種の「記号」だったのである。,これは妖怪にかぎったことではなく、あらゆる自然物がなんらかの意味を帯びた「記号」として存在し...,「物」は物そのものと言うよりも「記号」であったのである。,これらの「記号」は所与のものとして存在しており、人間にできるのはその「記号」を「読み取る」こ...,１２「物」が同時に「言葉」を伝える「記号」である世界。,こうした認識は、しかし近世において大きく変容する。
